In [1]:
!rm -rf ctcdecode
%cd /content
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .
%cd /content

/content
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14057, done.        
remote: Counting objects: 100% (370/370), done.        
remote: Compressing objects: 100% (299/299), done.        
remote: Total 14057 (

In [2]:
!pip install -q torchaudio
!pip install -q transformers
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic
!pip install ctc-segmentation

     |████████████████████████████████| 3.4 MB 5.6 MB/s 
     |████████████████████████████████| 895 kB 27.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
     |████████████████████████████████| 596 kB 19.4 MB/s 
     |████████████████████████████████| 61 kB 336 kB/s 
     |████████████████████████████████| 298 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 243 kB 43.5 MB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 192 kB 43.0 MB/s 
     |████████████████████████████████| 271 kB 47.3 MB/s 
     |████████████████████████████████| 160 kB 48.0 MB/s 
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 126 kB 7.1 MB/s 
     |████████████████████████████████| 71 kB 3.1 MB/s 
  Created wheel for ctc-segmentation: filename=ctc_segmentation-1.7.1-cp37-cp37m-linux_x86_64.whl size=114789 sha256=ea6e6b36e61b30f414da39

In [3]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
import ctc_segmentation
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
# from trainer import CTCTrainer
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
# from models import Wav2Vec2ClassificationModel
# from processors import CustomWav2Vec2Processor
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
# from arg_parsers import DataTrainingArguments, ModelArguments, DataCollatorCTCWithPadding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [4]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [5]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [6]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [7]:
dataset_wav_folder = 'dataset_wav'

In [8]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [9]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


## KenLM Wrapper Classes

In [11]:
#!/usr/bin/env python
# ----------------------------------------------------------------------------
# Copyright 2015-2016 Nervana Systems Inc.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ----------------------------------------------------------------------------
# Modified to support pytorch Tensors

import Levenshtein as Lev
import torch
from six.moves import xrange


class Decoder(object):
    """
    Basic decoder class from which all other decoders inherit. Implements several
    helper functions. Subclasses should implement the decode() method.
    Arguments:
        labels (list): mapping from integers to characters.
        blank_index (int, optional): index for the blank '_' character. Defaults to 0.
        space_index (int, optional): index for the space ' ' character. Defaults to 28.
    """

    def __init__(self, labels, blank_index=0):
        self.labels = labels
        self.int_to_char = dict([(i, c) for (i, c) in enumerate(labels)])
        self.blank_index = blank_index
        space_index = len(labels)  # To prevent errors in decode, we add an out of bounds index for the space
        if ' ' in labels:
            space_index = labels.index(' ')
        self.space_index = space_index

    def wer(self, s1, s2):
        """
        Computes the Word Error Rate, defined as the edit distance between the
        two provided sentences after tokenizing to words.
        Arguments:
            s1 (string): space-separated sentence
            s2 (string): space-separated sentence
        """

        # build mapping of words to integers
        b = set(s1.split() + s2.split())
        word2char = dict(zip(b, range(len(b))))

        # map the words to a char array (Levenshtein packages only accepts
        # strings)
        w1 = [chr(word2char[w]) for w in s1.split()]
        w2 = [chr(word2char[w]) for w in s2.split()]

        return Lev.distance(''.join(w1), ''.join(w2))

    def cer(self, s1, s2):
        """
        Computes the Character Error Rate, defined as the edit distance.
        Arguments:
            s1 (string): space-separated sentence
            s2 (string): space-separated sentence
        """
        s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
        return Lev.distance(s1, s2)

    def decode(self, probs, sizes=None):
        """
        Given a matrix of character probabilities, returns the decoder's
        best guess of the transcription
        Arguments:
            probs: Tensor of character probabilities, where probs[c,t]
                            is the probability of character c at time t
            sizes(optional): Size of each sequence in the mini-batch
        Returns:
            string: sequence of the model's best guess for the transcription
        """
        raise NotImplementedError


class BeamCTCDecoder(Decoder):
    def __init__(self, labels, lm_path=None, alpha=0, beta=0, cutoff_top_n=40, cutoff_prob=1.0, beam_width=100,
                 num_processes=4, blank_index=0):
        super(BeamCTCDecoder, self).__init__(labels)
        try:
            from ctcdecode import CTCBeamDecoder
        except ImportError:
            raise ImportError("BeamCTCDecoder requires paddledecoder package.")
        labels = list(labels)  # Ensure labels are a list before passing to decoder
        self._decoder = CTCBeamDecoder(labels, lm_path, alpha, beta, cutoff_top_n, cutoff_prob, beam_width,
                                       num_processes, blank_index, log_probs_input=True)

    def convert_to_strings(self, out, seq_len):
        results = []
        for b, batch in enumerate(out):
            utterances = []
            for p, utt in enumerate(batch):
                size = seq_len[b][p]
                if size > 0:
                    transcript = ''.join(map(lambda x: self.int_to_char[x.item()], utt[0:size]))
                else:
                    transcript = ''
                utterances.append(transcript)
            results.append(utterances)
        return results

    def convert_tensor(self, offsets, sizes):
        results = []
        for b, batch in enumerate(offsets):
            utterances = []
            for p, utt in enumerate(batch):
                size = sizes[b][p]
                if sizes[b][p] > 0:
                    utterances.append(utt[0:size])
                else:
                    utterances.append(torch.tensor([], dtype=torch.int))
            results.append(utterances)
        return results

    def decode(self, probs, sizes=None):
        """
        Decodes probability output using ctcdecode package.
        Arguments:
            probs: Tensor of character probabilities, where probs[c,t]
                            is the probability of character c at time t
            sizes: Size of each sequence in the mini-batch
        Returns:
            string: sequences of the model's best guess for the transcription
        """
        probs = probs.cpu()
        out, scores, offsets, seq_lens = self._decoder.decode(probs, sizes)
        print(out,scores,offsets,seq_lens)
        strings = self.convert_to_strings(out, seq_lens)
        offsets = self.convert_tensor(offsets, seq_lens)
        return strings, offsets


class GreedyDecoder(Decoder):
    def __init__(self, labels, blank_index=0):
        super(GreedyDecoder, self).__init__(labels, blank_index)

    def convert_to_strings(self, sequences, sizes=None, remove_repetitions=False, return_offsets=False):
        """Given a list of numeric sequences, returns the corresponding strings"""
        strings = []
        offsets = [] if return_offsets else None
        for x in xrange(len(sequences)):
            seq_len = sizes[x] if sizes is not None else len(sequences[x])
            string, string_offsets = self.process_string(sequences[x], seq_len, remove_repetitions)
            strings.append([string])  # We only return one path
            if return_offsets:
                offsets.append([string_offsets])
        if return_offsets:
            return strings, offsets
        else:
            return strings

    def process_string(self, sequence, size, remove_repetitions=False):
        string = ''
        offsets = []
        for i in range(size):
            char = self.int_to_char[sequence[i].item()]
            if char != self.int_to_char[self.blank_index]:
                # if this char is a repetition and remove_repetitions=true, then skip
                if remove_repetitions and i != 0 and char == self.int_to_char[sequence[i - 1].item()]:
                    pass
                elif char == self.labels[self.space_index]:
                    string += ' '
                    offsets.append(i)
                else:
                    string = string + char
                    offsets.append(i)
        return string, torch.tensor(offsets, dtype=torch.int)

    def decode(self, probs, sizes=None):
        """
        Returns the argmax decoding given the probability matrix. Removes
        repeated elements in the sequence, as well as blanks.
        Arguments:
            probs: Tensor of character probabilities from the network. Expected shape of batch x seq_length x output_dim
            sizes(optional): Size of each sequence in the mini-batch
        Returns:
            strings: sequences of the model's best guess for the transcription on inputs
            offsets: time step per character predicted
        """
        _, max_probs = torch.max(probs, 2)
        strings, offsets = self.convert_to_strings(max_probs.view(max_probs.size(0), max_probs.size(1)), sizes,
                                                   remove_repetitions=True, return_offsets=True)
        return strings, offsets

In [12]:
class Transcriber:
    def __init__(self, model_path, processor_path="bakrianoo/sinai-voice-ar-stt"):
        self.processor = Wav2Vec2Processor.from_pretrained(processor_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()
        self.model.to("cuda")

    def transcribe(self, file_path, for_lm_pred = True):
        speech, sampling_rate = librosa.load(file_path, sr=16_000)
        inputs = self.processor(
            speech, sampling_rate=16000, return_tensors="pt", padding=True
        )
        with torch.no_grad():
          logits = self.model(inputs.input_values.to('cuda')).logits
          for logit in logits:
            logit[logit==-100] = self.processor.tokenizer.pad_token_id
          if for_lm_pred:
            return logits, inputs.input_values.shape[1]
          predicted = torch.argmax(logits, dim=-1)
        # predicted[predicted == -100] = self.processor.tokenizer.pad_token_id
        predicted_tokenized = self.processor.tokenizer.batch_decode(predicted)
        return predicted_tokenized

def get_segments(logits, decoded_output, max_signal_length, sampling_rate, vocab_list):
    # CTC log posteriors inference
    with torch.no_grad():
        softmax = torch.nn.LogSoftmax(dim=-1)
        lpzs = softmax(logits).cpu().numpy()

    batch_segments_list = []
    # for i in range(len(decoded_output)):
    # lpz = lpzs[i] # select the logits of ith file
    lpz = lpzs
    transcription = decoded_output[0] # 0 means the most probable transcription
    text = transcription.split()
    print(text)
    
    # CTC segmentation preparation
    config = ctc_segmentation.CtcSegmentationParameters(char_list=vocab_list)
    config.index_duration = max_signal_length / lpz.shape[0] / sampling_rate
    print(config.index_duration)
    
    # CTC segmentation
    ground_truth_mat, utt_begin_indices = ctc_segmentation.prepare_text(config, text)
    timings, char_probs, state_list = ctc_segmentation.ctc_segmentation(config, lpz, ground_truth_mat)
    segments = ctc_segmentation.determine_utterance_segments(config, utt_begin_indices, char_probs, timings, text)
    
    segments_list = []
    for word, segment in zip(text, segments):
        start_time, end_time, min_avg = segment
        segment_dict = {
          'start_time': start_time,
          'end_time': end_time,
          'duration': (end_time - start_time),
          'word': word
        }
        segments_list.append(segment_dict)
    # batch_segments_list.append(segments_list)
    # return batch_segments_list
    return segments_list


In [8]:
# !sed -i 's/MODEL_ID = "jonatasgrosman\/wav2vec2-large-xlsr-53-english"/MODEL_ID = "bakrianoo\/sinai-voice-ar-stt"/' wav2vec2_kenlm.py
# !sed -i 's/lm_path = "data\/local\/local_lm\/data\/arpa\/4gram_big.arpa"/lm_path = "\/content\/drive\/MyDrive\/KFUPM-Master\/ICS606\/Models\/baits_dataset_kenlm_model.arpa"/' wav2vec2_kenlm.py

In [13]:
lm_path = "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/baits_dataset_kenlm_model.arpa"

In [14]:
MODEL_ID = "bakrianoo/sinai-voice-ar-stt"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [15]:
vocab_dict = processor.tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
vocab = []
for _, token in sort_vocab:
    vocab.append(token)
len(vocab)

48

In [16]:
alpha=0
beta=0
beam_width = 10
beam_decoder = BeamCTCDecoder(
    vocab,
    lm_path=lm_path,
    alpha=alpha,
    beta=beta,
    cutoff_top_n=100,
    cutoff_prob=1.0,
    beam_width=beam_width,
    num_processes=16,
    blank_index=vocab.index(processor.tokenizer.pad_token),
)

In [17]:
transcriber = Transcriber(model_path=MODEL_ID)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
transcriber.transcribe(f'{dataset_wav_folder}/{test_metadata["Utterance name"][20]}',for_lm_pred=False)

['صليت مني نذيل بخرط لا يمل الشر حتى يمل']

In [19]:
test_metadata["Clean Script"][20]

'صليت مني هذيل بخرق      لا يمل الشر حتى يملوا '

In [20]:
logits, max_signal_length = transcriber.transcribe(f'{dataset_wav_folder}/{test_metadata["Utterance name"][20]}')
max_signal_length

112640

In [36]:
beam_decoder._decoder.decode(logits)[2]

tensor([[[         29,          37,          42,  ..., -1113457961,
          -1127798841, -1115582246],
         [         29,          37,          42,  ..., -1110955024,
          -1120549871, -1118882618],
         [         29,          37,          42,  ..., -1111435727,
          -1121921799, -1123139891],
         ...,
         [         29,          37,          42,  ..., -1117372259,
          -1126726091,  1003626963],
         [         29,          37,          42,  ..., -1116781240,
          -1111954095, -1123236009],
         [         29,          37,          42,  ..., -1121713267,
          -1124610303, -1134161426]]], dtype=torch.int32)

In [24]:
beam_decoded_output, beam_decoded_offsets = beam_decoder.decode(logits)
beam_decoded_output = beam_decoded_output[0]

tensor([[[        21,         34,         33,  ...,  543516788,
          1635017028, 1835102790],
         [        21,         34,         33,  ..., 1701601889,
          1717924384, 1852142181],
         [        21,         34,         33,  ..., 1380393018,
           542135631, 1847613762],
         ...,
         [        21,         34,         33,  ..., 1714564656,
           153711485, 1769211098],
         [        21,         34,         33,  ...,  142224996,
          1634741549, 1936548210],
         [        21,         34,         33,  ..., 1914730866,
          1819636581,  170816372]]], dtype=torch.int32) tensor([[-3924.7334, -3923.8169, -3922.9890, -3922.8992, -3922.6626, -3922.1746,
         -3921.6907, -3921.1938, -3920.7727, -3920.7498]]) tensor([[[         29,          37,          42,  ..., -1116427549,
          -1114544831, -1125910150],
         [         29,          37,          42,  ...,  1016808439,
          -1127607705,  1025992449],
         [         29

In [25]:
beam_decoded_output

['صليتمنينذيلبخرطلايملالشرحتىيمل',
 'صليتمنيهذيلبخرطلايملالشرحتىيمل',
 'صليتمنيلذيلبخرطلايملالشرحتىيمل',
 'صليتمنينذيلبخلطلايملالشرحتىيمل',
 'صليتمنينذيلبخرقلايملالشرحتىيمل',
 'صليتمنينذيلبخرطلايملالشرحتىايمل',
 'صليتمنينذيلبخرطلايملالشرحتىيملو',
 'صليتمنيهذيلبخرطلايملالشرحتىايمل',
 'صليتمنيهذيلبخرطلايملالشرحتىيملو',
 'صليتمنينذيلبخرطلايملالشرحتىيملا']

In [41]:
get_segments(logits, beam_decoded_output, max_signal_length, 16_000, vocab)

[]
5.9946875


AssertionError: ignored